## Problem Analysis

## Data Collection

### Importing libraries

In [94]:
# Import all necessary libraries
import pandas as pd
import ydata_profiling
import os
import numpy as np
import gc
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import mlflow

# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

### Collecting data

In [23]:
# Paso 1: Ingesta y unificación de datos con polars

# Rutas de los archivos
setA_path = r"C:\repos\physionet-sepsis-forecasting\data\raw\all_patients_setA.parquet"
setB_path = r"C:\repos\physionet-sepsis-forecasting\data\raw\all_patients_setB.parquet"
unified_path = r"C:\repos\physionet-sepsis-forecasting\data\raw\all_patients_unified.parquet"

In [33]:
# Leer ambos datasets con pandas
df_a = pd.read_parquet(setA_path)
df_b = pd.read_parquet(setB_path)

In [34]:
# Mostrar las columnas que NO tienen en común
print("Columnas en Set A pero no en Set B:", set(df_a.columns) - set(df_b.columns))
print("Columnas en Set B pero no en Set A:", set(df_b.columns) - set(df_a.columns))

Columnas en Set A pero no en Set B: {'subHR'}
Columnas en Set B pero no en Set A: set()


In [35]:
# Asegurar que ambos tengan las mismas columnas eliminando las que no coinciden
common_cols = list(set(df_a.columns) & set(df_b.columns))
df_a = df_a[common_cols]
df_b = df_b[common_cols]

In [36]:
# Unificar
df = pd.concat([df_a, df_b])

# Guardar el dataset unificado
df.to_parquet(unified_path)
print(f"Dataset unificado guardado en {unified_path}")

Dataset unificado guardado en C:\repos\physionet-sepsis-forecasting\data\raw\all_patients_unified.parquet


In [37]:
# Limpiar memoria eliminando df_a y df_b
del df_a
del df_b
gc.collect()

1714

## EDA

In [64]:
# Paso 2: Análisis exploratorio
df = pd.read_parquet(unified_path)
df.shape

(1552210, 42)

In [39]:
# Ver las primeras filas
df.head()

,Chloride,Unit1,Lactate,Bilirubin_total,Glucose,Creatinine,SBP,Temp,WBC,HospAdmTime,TroponinI,HR,PTT,SepsisLabel,Potassium,Age,HCO3,Alkalinephos,BaseExcess,Hgb,FiO2,Calcium,Phosphate,Resp,EtCO2,MAP,Gender,Platelets,Fibrinogen,patient_id,Magnesium,SaO2,Hct,BUN,ICULOS,DBP,pH,Unit2,Bilirubin_direct,PaCO2,AST,O2Sat
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,NaN,NaN,0.0,NaN,83.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,p000001,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,98.0,NaN,NaN,-0.03,NaN,97.0,NaN,0.0,NaN,83.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,NaN,75.33,0.0,NaN,NaN,p000001,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,95.0
2,NaN,NaN,NaN,NaN,NaN,NaN,122.0,NaN,NaN,-0.03,NaN,89.0,NaN,0.0,NaN,83.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,86.00,0.0,NaN,NaN,p000001,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,99.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,90.0,NaN,0.0,NaN,83.14,NaN,NaN,24.0,NaN,NaN,NaN,NaN,30.0,NaN,NaN,0.0,NaN,NaN,p000001,NaN,NaN,NaN,NaN,4.0,NaN,7.36,NaN,NaN,100.0,NaN,95.0
4,NaN,NaN,NaN,NaN,NaN,NaN,122.0,NaN,NaN,-0.03,NaN,103.0,NaN,0.0,NaN,83.14,NaN,NaN,NaN,NaN,0.28,NaN,NaN,24.5,NaN,91.33,0.0,NaN,NaN,p000001,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,88.5


In [40]:
# Descripción estadística rápida
df.describe()

,Chloride,Unit1,Lactate,Bilirubin_total,Glucose,Creatinine,SBP,Temp,WBC,HospAdmTime,TroponinI,HR,PTT,SepsisLabel,Potassium,Age,HCO3,Alkalinephos,BaseExcess,Hgb,FiO2,Calcium,Phosphate,Resp,EtCO2,MAP,Gender,Platelets,Fibrinogen,Magnesium,SaO2,Hct,BUN,ICULOS,DBP,pH,Unit2,Bilirubin_direct,PaCO2,AST,O2Sat
count,70466.000000,940250.000000,41446.000000,23141.000000,265516.000000,94616.000000,1.325945e+06,525226.000000,99447.000000,1.552202e+06,14781.000000,1.398786e+06,45699.000000,1.552210e+06,144525.000000,1.552210e+06,65028.000000,24941.000000,84145.000000,114591.000000,129365.000000,91331.000000,62301.000000,1.313875e+06,57636.000000,1.358940e+06,1.552210e+06,92209.000000,10242.000000,97951.000000,53561.000000,137433.000000,106568.000000,1.552210e+06,1.065656e+06,107573.000000,940250.000000,2990.000000,86301.000000,25183.000000,1.349474e+06
mean,105.827910,0.496571,2.646666,2.114059,136.932283,1.510699,1.237505e+02,36.977228,11.446405,-5.612512e+01,8.290099,8.458154e+01,41.231193,1.798468e-02,4.135528,6.200947e+01,24.075481,102.483661,-0.689919,10.430833,0.554839,7.557531,3.544238,1.872650e+01,32.957657,8.240010e+01,5.592690e-01,196.013911,287.385706,2.051450,92.654188,30.794093,23.915452,2.699499e+01,6.383056e+01,7.378934,0.503429,1.836177,41.021869,260.223385,9.719395e+01
std,5.880462,0.499989,2.526214,4.311468,51.310728,1.805603,2.323156e+01,0.770014,7.731013,1.622569e+02,24.806235,1.732537e+01,26.217669,1.328956e-01,0.642150,1.638622e+01,4.376504,120.122746,4.294297,1.968661,11.123207,2.433152,1.423286,5.098194e+00,7.951662,1.634175e+01,4.964749e-01,103.635366,153.002908,0.397898,10.892986,5.491749,19.994317,2.900542e+01,1.395601e+01,0.074568,0.499989,3.694082,9.267242,855.746795,2.936924e+00
min,26.000000,0.000000,0.200000,0.100000,10.000000,0.100000,2.000000e+01,20.900000,0.100000,-5.366860e+03,0.010000,2.000000e+01,12.500000,0.000000e+00,1.000000,1.400000e+01,0.000000,7.000000,-32.000000,2.200000,-50.000000,1.000000,0.200000,1.000000e+00,10.000000,2.000000e+01,0.000000e+00,1.000000,34.000000,0.200000,23.000000,5.500000,1.000000,1.000000e+00,2.000000e+01,6.620000,0.000000,0.010000,10.000000,3.000000,2.000000e+01
25%,102.000000,0.000000,1.260000,0.500000,106.000000,0.700000,1.070000e+02,36.500000,7.600000,-4.705000e+01,0.040000,7.200000e+01,27.800000,0.000000e+00,3.700000,5.168000e+01,22.000000,54.000000,-3.000000,9.100000,0.400000,7.700000,2.600000,1.500000e+01,28.000000,7.100000e+01,0.000000e+00,126.000000,184.000000,1.800000,94.000000,27.000000,12.000000,1.100000e+01,5.400000e+01,7.340000,0.000000,0.200000,35.000000,22.000000,9.600000e+01
50%,106.000000,0.000000,1.800000,0.900000,127.000000,0.940000,1.210000e+02,37.000000,10.300000,-6.030000e+00,0.300000,8.350000e+01,32.400000,0.000000e+00,4.100000,6.400000e+01,24.000000,74.000000,0.000000,10.300000,0.500000,8.300000,3.300000,1.800000e+01,33.000000,8.000000e+01,1.000000e+00,181.000000,250.000000,2.000000,97.000000,30.300000,17.000000,2.100000e+01,6.200000e+01,7.380000,1.000000,0.445000,40.000000,41.000000,9.800000e+01
75%,109.000000,1.000000,3.000000,1.700000,153.000000,1.430000,1.380000e+02,37.500000,13.800000,-4.000000e-02,3.980000,9.550000e+01,42.800000,0.000000e+00,4.400000,7.400000e+01,26.800000,108.000000,1.000000,11.700000,0.600000,8.700000,4.100000,2.150000e+01,38.000000,9.200000e+01,1.000000e+00,244.000000,349.000000,2.200000,98.000000,34.100000,28.000000,3.400000e+01,7.200000e+01,7.430000,1.000000,1.700000,45.000000,111.000000,9.950000e+01
max,145.000000,1.000000,31.000000,49.600000,988.000000,46.600000,3.000000e+02,50.000000,440.000000,2.399000e+01,440.000000,2.800000e+02,250.000000,1.000000e+00,27.500000,1.000000e+02,55.000000,3833.000000,100.000000,32.000000,4000.000000,27.900000,18.800000,1.000000e+02,100.000000,3.000000e+02,1.000000e+00,2322.000000,1760.000000,9.800000,100.000000,71.700000,268.000000,3.360000e+02,3.000000e+02,7.930000,1.000000,37.500000,100.000000,9961.000000,1.000000e+02


In [44]:
# Conteo de valores nulos por columna con porcentaje
null_counts = df.isnull().sum()
null_percent = (null_counts / len(df)) * 100
null_df = pd.DataFrame({'null_count': null_counts, 'null_percent': null_percent})
null_df = null_df[null_df['null_count'] >= 0].sort_values(by='null_percent', ascending=False)
print(null_df)

                  null_count  null_percent
Bilirubin_direct     1549220     99.807371
Fibrinogen           1541968     99.340167
TroponinI            1537429     99.047745
Bilirubin_total      1529069     98.509158
Alkalinephos         1527269     98.393194
AST                  1527027     98.377604
Lactate              1510764     97.329872
PTT                  1506511     97.055875
SaO2                 1498649     96.549372
EtCO2                1494574     96.286843
Phosphate            1489909     95.986303
HCO3                 1487182     95.810618
Chloride             1481744     95.460279
BaseExcess           1468065     94.579020
PaCO2                1465909     94.440121
Calcium              1460879     94.116067
Platelets            1460001     94.059502
Creatinine           1457594     93.904433
Magnesium            1454259     93.689578
WBC                  1452763     93.593199
BUN                  1445642     93.134434
pH                   1444637     93.069688
Hgb        

In [47]:
# Analizar desbalance de clases en la variable objetivo SepsisLabel
sepsis_counts = df['SepsisLabel'].value_counts()
sepsis_percent = (sepsis_counts / len(df)) * 100
sepsis_classes = pd.DataFrame({'sepsis_counts': sepsis_counts, 'sepsis_percent': sepsis_percent})
sepsis_classes = sepsis_classes[sepsis_classes['sepsis_counts'] >= 0].sort_values(by='sepsis_percent', ascending=False)

print(sepsis_classes)

             sepsis_counts  sepsis_percent
SepsisLabel                               
0.0                1524294       98.201532
1.0                  27916        1.798468


In [48]:
# Generar un reporte con ydata-profiling

profile = ydata_profiling.ProfileReport(df, title="Reporte de Análisis Exploratorio", explorative=True, minimal=True)
# Si el directorio no existe, crearlo
os.makedirs(os.path.dirname(r"C:\repos\physionet-sepsis-forecasting\data\reports"), exist_ok=True)
# Guardar el reporte
profile_path = r"C:\repos\physionet-sepsis-forecasting\data\reports\eda_report.html"
profile.to_file(profile_path)

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 26.31it/s]


## Data Cleaning & Preprocessing

### Imputation

In [65]:
# Imputar valores nulos con la media usando pandas y para las variables categóricas con la moda
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for col in numeric_cols:
    mean_value = df[col].mean()
    df[col] = df[col].fillna(mean_value)

categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
for col in categorical_cols:
    mode_value = df[col].mode()[0]
    df[col] = df[col].fillna(mode_value)

### Data Scalation

In [66]:
# Aplicar StandardScaler de sklearn a las columnas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Excluir columnas que no deben ser escaladas
exclude_cols = ['SepsisLabel', 'patient_id', 'ICULOS']
numeric_cols = [col for col in numeric_cols if col not in exclude_cols]
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

## Modeling a LSTM

Transformamos el dataframe (que tiene una fila por hora por paciente) en un formato (número_de_muestras, longitud_de_secuencia, número_de_características), que es lo que un LSTM espera.

In [67]:
# Ordenamos y agrupamos por paciente
df = df.sort_values(by=['patient_id', 'ICULOS'])

### Define features and target

In [68]:
# Denifimos características y etiquetas
features_cols = [col for col in df.columns if col not in ['SepsisLabel', 'patient_id']]
target_col = 'SepsisLabel'

### Sliding windows

In [69]:
# Calcular la secuencia minima y máxima por paciente
seq_lengths = df.groupby('patient_id').size()
min_seq_length = seq_lengths.min()
max_seq_length = seq_lengths.max() 
print(f"Longitud mínima de secuencia por paciente: {min_seq_length}")
print(f"Longitud máxima de secuencia por paciente: {max_seq_length}")

Longitud mínima de secuencia por paciente: 8
Longitud máxima de secuencia por paciente: 336


In [70]:
# Contar cuantos pacientes hay con la secuencia mínima y máxima
min_seq_count = (seq_lengths == min_seq_length).sum()
max_seq_count = (seq_lengths == max_seq_length).sum()
print(f"Número de pacientes con la secuencia mínima ({min_seq_length}): {min_seq_count}")
print(f"Número de pacientes con la secuencia máxima ({max_seq_length}): {max_seq_count}")

Número de pacientes con la secuencia mínima (8): 328
Número de pacientes con la secuencia máxima (336): 10


In [ ]:
# Creamos secuencias para LSTM (Ventanas deslizantes)
# Parámetros
sequence_length = min_seq_length # Usar la secuencia minima de datos como para predecir la siguiente
X_sequences = []
y_sequences = []

# Agrupar por paciente para no mezclar datos de diferentes personas
grouped = df.groupby('patient_id')

for _, group in grouped:
    features = group[features_cols].values
    target = group[target_col].values
    
    # Crear ventanas deslizantes para cada paciente
    for i in range(len(group) - sequence_length):
        X_sequences.append(features[i:i + sequence_length])
        y_sequences.append(target[i + sequence_length])

# Convertir a arrays de NumPy
X = np.array(X_sequences)
y = np.array(y_sequences)

print(f"Forma de las secuencias de entrada (X): {X.shape}")
print(f"Forma de las etiquetas de salida (y): {y.shape}")

# La salida de X.shape debería ser (num_muestras, min_seq_length, num_features)

Forma de las secuencias de entrada (X): (1229522, 8, 40)
Forma de las etiquetas de salida (y): (1229522,)


### Split in Train & Test

In [74]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Train shapes: {X_train.shape}, {y_train.shape}")
print(f"Validation shapes: {X_val.shape}, {y_val.shape}")
print(f"Test shapes: {X_test.shape}, {y_test.shape}")

Train shapes: (860665, 8, 40), (860665,)
Validation shapes: (184428, 8, 40), (184428,)
Test shapes: (184429, 8, 40), (184429,)


### Save processed data

Ahora que tenemos los datos en el formato correcto los guardaremos en la carpeta `data\processed`. No guardaremos un .parquet porque ya no es un dataframe 2D. Usaremos el formato de NumPy (.npy) que es ideal para arrays multidimensionales.

In [82]:
processed_dir = r"C:\repos\physionet-sepsis-forecasting\data\processed"
os.makedirs(processed_dir, exist_ok=True)

np.save(os.path.join(processed_dir, 'X_train.npy'), X_train)
np.save(os.path.join(processed_dir, 'y_train.npy'), y_train)
np.save(os.path.join(processed_dir, 'X_val.npy'), X_val)
np.save(os.path.join(processed_dir, 'y_val.npy'), y_val)
np.save(os.path.join(processed_dir, 'X_test.npy'), X_test)
np.save(os.path.join(processed_dir, 'y_test.npy'), y_test)

print("Datos procesados y divididos guardados en data/processed/")

Datos procesados y divididos guardados en data/processed/


### Versionamiento con DVC:
Ahora, añade estos archivos .npy a DVC, tal como hiciste con el archivo Parquet, y sube los cambios.

In [ ]:
dvc add data/processed
git add data/processed.dvc
git commit -m "feat: Create time-series sequences for LSTM model"
dvc push

### Fase 2: Implementación y Entrenamiento del Modelo LSTM

**Objetivo**: Construir la arquitectura del modelo, entrenarlo con los datos secuenciales y registrar los resultados con MLflow.

2.1. Adaptar el Notebook para el Entrenamiento (o crear scripts/train.py):
Te recomiendo encarecidamente mover el código de entrenamiento del notebook a un script scripts/train.py para seguir las buenas prácticas del repositorio.
2.2. Crear los DataLoaders de PyTorch:
PyTorch usa DataLoader para gestionar los datos en batches de manera eficiente.

In [ ]:
# Liberar memoria de las variables originales
del X_train, y_train, X_val, y_val, X_test, y_test, df, X_temp, y_temp
gc.collect()

405572

In [92]:
# Cargar los datos desde data/processed
processed_dir = r"C:\repos\physionet-sepsis-forecasting\data\processed"
X_train = np.load(os.path.join(processed_dir, 'X_train.npy'))
y_train = np.load(os.path.join(processed_dir, 'y_train.npy'))
X_val = np.load(os.path.join(processed_dir, 'X_val.npy'))
y_val = np.load(os.path.join(processed_dir, 'y_val.npy'))

# Convertir a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)

# Crear TensorDatasets y DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

### 2.3. Definir la Arquitectura del Modelo LSTM:
Aquí definimos las capas del modelo.

In [93]:
class SepsisLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(SepsisLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Inicializar estados ocultos
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Pasar por el LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Tomar la salida del último paso de tiempo y pasarla por la capa densa
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

### 2.4. Bucle de Entrenamiento y Evaluación:
Este es el corazón del entrenamiento, donde iteramos sobre los datos, calculamos la pérdida y ajustamos los pesos del modelo.

In [ ]:
# Configuración del modelo y entrenamiento
input_size = X_train.shape[2] # Número de features
hidden_size = 128
num_layers = 2
output_size = 1
num_epochs = 10
learning_rate = 0.001

model = SepsisLSTM(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss() # Binary Cross-Entropy para clasificación binaria
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# --- INTEGRACIÓN CON MLFLOW ---
mlflow.set_tracking_uri("http://13.222.205.81:8050")
with mlflow.start_run() as run:
    mlflow.log_params({"hidden_size": hidden_size, "num_layers": num_layers, "epochs": num_epochs})

for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Bucle de validación (calcular métricas como AUC-ROC en el val_loader)
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
    
    auc = roc_auc_score(all_labels, all_preds)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val AUC: {auc:.4f}")
    mlflow.log_metric("val_auc", auc, step=epoch)

    mlflow.pytorch.log_model(model, "model")